In [ ]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer

import phd

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def img(array, zscore=False):
    if zscore:
        array = stats.zscore(array, axis=0)
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

def img_compare(array1, array2, zscore=(False, False)):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    img(array1, zscore[0])
    plt.subplot(1, 2, 2)
    img(array2, zscore[1])
    plt.tight_layout()

timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

In [ ]:
# A peek into the neural implementation
plt.figure(figsize=(8, 14))
plt.subplot(3, 1, 1)
phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), model.freqs)
plt.subplot(3, 1, 2)
phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), model.freqs)
plt.subplot(3, 1, 3)
rasterplot(sim.trange(), sim.data[an_p])
plt.ylim(0, net.periphery.an.n_neurons * net.periphery.an.n_ensembles)
sns.despine()
plt.tight_layout()

## With derivatives

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]
model.add_derivative()  # First derivative
model.add_derivative()  # Second derivative

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

## With Adaptive LIF neurons in AN

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]
model.periphery.adaptive_neurons = True

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

## Running a classification experiment

In [ ]:
# No derivatives
model = phd.sermo.AuditoryFeatures()
expt = phd.experiments.AuditoryFeaturesExperiment(model, phones=phd.timit.TIMIT.phones)
expt.seed = 9
expt.timit.filefilt.region = 8
expt.cache_key

In [ ]:
# With 2 derivatives
model = phd.sermo.AuditoryFeatures()
model.add_derivative(klass="FeedforwardDeriv")
model.add_derivative(klass="FeedforwardDeriv")
expt = phd.experiments.AuditoryFeaturesExperiment(model, seed=0, phones=phd.timit.TIMIT.consonants)
expt.timit.filefilt.region = 1
key = expt.run()
result = phd.experiments.AuditoryFeaturesResult.load(key)
print "Accuracy with MFCC:", result.mfcc_acc
print "Accuracy with NCC:", result.ncc_acc

In [ ]:
# With 1 derivative and z-score
model = phd.sermo.AuditoryFeatures()
model.add_derivative(klass="FeedforwardDeriv")
expt = phd.experiments.AuditoryFeaturesExperiment(model, seed=0, phones=phd.timit.TIMIT.consonants)
expt.zscore = True
expt.timit.filefilt.region = 1
key = expt.run()
result = phd.experiments.AuditoryFeaturesResult.load(key)
print "Accuracy with MFCC:", result.mfcc_acc
print "Accuracy with NCC:", result.ncc_acc

## Plotting experimental results

In [ ]:
# Put figures in phd/figures
root = os.path.abspath(os.path.join(os.path.dirname(phd.__file__), ".."))
def fig(name, ext='svg'):
    return os.path.join(root, "figures", "results", "%s.%s" % (name, ext))

def accuracy_plots(columns, vary, hue_order, relative=True, filter_by=None):
    df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, columns + ['phones'])
    filter_by = [('phones', 'consonants')] if filter_by is None else filter_by
    phones = 'Consonants'

    common_args = {'x_label': 'Feature',
                   'y_label': 'Accuracy',
                   'hue_order': hue_order,
                   'filter_by': filter_by,
                   'group_by': vary}

    if len(filter_by) > 0:
        for k, v in filter_by:
            if k == 'phones':
                phones = v
                common_args['y_label'] = '%s accuracy' % phones[:-1].capitalize()
    

    for plot_f in [sns.violinplot, sns.barplot]:
        acc = plt.figure()
        if relative:
            phd.plots.compare(df,
                              columns=['ncc_train_acc', 'ncc_test_acc'],
                              relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                              x_keys=['Training', 'Testing'],
                              plot_f=plot_f,
                              **common_args)
            plt.axhline(1.0, c='k', ls=':')
            plt.ylabel('Relative %s accuracy' % phones[:-1].lower())
        else:
            phd.plots.compare(df,
                              columns=['mfcc_train_acc', 'mfcc_test_acc', 'ncc_train_acc', 'ncc_test_acc'],
                              x_keys=['MFCC training', 'MFCC testing', 'NCC training', 'NCC testing'],
                              plot_f=plot_f,
                              **common_args)
            plt.ylabel('%s accuracy' % phones[:-1].capitalize())
        plt.xlabel("")
        acc.savefig(fig('ncc-%s-%sacc-%s' % (
            vary, 'r' if relative else '', 'v' if plot_f is sns.violinplot else 'b')))

def accuracy_tsplots(columns, vary, relative=True, filter_by=None):
    df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, columns + ['phones'])
    filter_by = [('phones', 'consonants')] if filter_by is None else filter_by
    phones = 'Consonants'

    common_args = {'x_label': 'Feature',
                   'y_label': 'Accuracy',
                   'filter_by': filter_by,
                   'group_by': vary}

    if len(filter_by) > 0:
        for k, v in filter_by:
            if k == 'phones':
                phones = v
                common_args['y_label'] = '%s accuracy' % v[:-1].capitalize()
    
    acc = plt.figure()
    if relative:
        phd.plots.timeseries(df,
                             columns=['ncc_train_acc', 'ncc_test_acc'],
                             relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                             x_keys=['Training', 'Testing'],
                             **common_args)
        plt.axhline(1.0, c='k', ls=':')
        plt.ylabel('Relative %s accuracy' % phones[:-1].lower())
    else:
        phd.plots.timeseries(df,
                             columns=['mfcc_train_acc', 'mfcc_test_acc', 'ncc_train_acc', 'ncc_test_acc'],
                             x_keys=['MFCC training', 'MFCC testing', 'NCC training', 'NCC testing'],
                             **common_args)
        plt.ylabel('%s accuracy' % phones[:-1].capitalize())
    plt.xlabel("")
    acc.savefig(fig('ncc-%s-acc-t' % vary))

def time_plot(columns, vary, hue_order, filter_by=None):
    df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, columns + ['phones'])
    filter_by = [('phones', 'consonants')] if filter_by is None else filter_by

    time = plt.figure()
    phd.plots.compare(df,
                      columns=['mfcc_time', 'ncc_time', 'mfcc_fit_time', 'ncc_fit_time'],
                      group_by=vary,
                      filter_by=filter_by,
                      x_keys=['MFCC generation', 'NCC generation', 'MFCC SVM fitting', 'NCC SVM fitting'],
                      x_label='Scenario',
                      y_label='Time (s)',
                      plot_f=sns.barplot,
                      hue_order=hue_order)
    plt.ylabel("Time (s)")
    plt.xlabel("")
    time.savefig(fig('ncc-%s-time' % vary))

In [ ]:
pargs = {'columns': [], 'vary': 'phones', 'filter_by': [], 'hue_order': ["vowels", "consonants", "all"]}

phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
accuracy_plots(relative=False, **pargs)
accuracy_plots(relative=True, **pargs)
plt.ylim(bottom=0.95)
time_plot(**pargs)

In [ ]:
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
accuracy_plots(['zscore'], 'zscore', hue_order=['False', 'True'], relative=False)

In [ ]:
pargs = {'columns': ['derivatives'], 'vary': 'derivatives', 'hue_order': ['0', '1', '2']}

phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
accuracy_plots(relative=False, **pargs)

In [ ]:
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc'
accuracy_plots(['deriv_type'], 'deriv_type',
               hue_order=['FeedforwardDeriv', 'IntermediateDeriv'])

In [ ]:
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
accuracy_tsplots(['periphery'], 'periphery')

In [ ]:
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
accuracy_tsplots(['feature'], 'feature')
time_plot(['feature'], 'feature',
          [str(x) for x in [1, 8, 12, 16, 32, 64]])

In [ ]:
pargs = {'columns': ['dt'], 'vary': 'dt', 'hue_order': ["0.010000", "0.005000", "0.001000"]}

phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
accuracy_plots(relative=False, **pargs)

In [ ]:
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_final'
columns = ['periphmodel', 'adaptive']
porder = ['gammatone', 'log_gammachirp', 'dual_resonance', 'compressive_gammachirp', 'tan_carney']
accuracy_plots(columns, 'periphmodel', hue_order=porder, relative=True)
plt.ylim(bottom=0.95)
time_plot(columns, 'periphmodel', hue_order=porder)
accuracy_plots(columns, 'adaptive', hue_order=['False', 'True'], relative=True)
plt.ylim(bottom=0.95);

## Scaling

In [ ]:
def n_neurons(msg, model):
    net = model.build()
    nn = sum(e.n_neurons for e in net.all_ensembles)
    print("=== %s ===" % msg)
    pneurons = model.freqs.size * model.periphery.neurons_per_freq
    print("Periphery layer: %d freqs x %d neurons_per_freq = %d neurons" % (
        model.freqs.size, model.periphery.neurons_per_freq, pneurons))
    cneurons = model.n_cepstra * model.cepstra.n_neurons
    print("Feature layer: %d cepstra x %d neurons_per_cepstra = %d neurons" % (
        model.n_cepstra, model.cepstra.n_neurons, cneurons))
    for i, deriv in enumerate(model.derivatives):
        # Note! Assumes FeedforwardDeriv!
        cneurons += model.n_cepstra * 2 * deriv.n_neurons
        print("Derivative %d: %d cepstra x 2 x %d neurons_per_cepstra = %d neurons" % (
            (i+1), model.n_cepstra, deriv.n_neurons, model.n_cepstra * deriv.n_neurons))

    assert pneurons + cneurons == nn
    print("Total: %d neurons" % nn)
    print("")

model = phd.sermo.AuditoryFeatures()
model.add_derivative()
n_neurons("Default configuration", model)
model = phd.sermo.AuditoryFeatures()
model.freqs = phd.filters.erbspace(20, 20000, 3500)
model.periphery.neurons_per_freq = 20
model.n_cepstra = 20
model.cepstra.n_neurons = 50
model.add_derivative(n_neurons=50)
n_neurons("Conservative estimate", model)
model = phd.sermo.AuditoryFeatures()
model.freqs = phd.filters.erbspace(20, 20000, 3500)
model.periphery.neurons_per_freq = 40
model.n_cepstra = 40
model.cepstra.n_neurons = 200
model.add_derivative(n_neurons=200)
model.add_derivative(n_neurons=200)
n_neurons("Generous estimate", model);